In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft


import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(20, folder='two_dim_majoranas')
hpc05.kill_remote_ipcluster()
# client, dview, lview = hpc05.start_remote_and_connect(20, folder='two_dim_majoranas')

In [ ]:
import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, spectrum, topology, supercurrent

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Make T shaped system

In [ ]:

def l_shaped_system(a, L_m, L_x, z_x, z_y, W_up, W_down, edge_thickness=1,
                    transverse_soi=True,
                    mu_from_bottom_of_spin_orbit_bands=True,
                    k_x_in_sc=False, leaded=True, **_):

    #     HAMILTONIAN DEFINITIONS
    template_strings = sns_system.get_template_strings(
        transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)
    
    conservation_matrix = -supercurrent.sigz
    
    # TURN HAMILTONIAN STRINGS INTO TEMPLATES
    kwargs = dict(coords=('x', 'y'), grid_spacing=a)
    template_barrier = kwant.continuum.discretize(
        template_strings['ham_barrier'], **kwargs)
    template_normal = kwant.continuum.discretize(
        template_strings['ham_normal'], **kwargs)
    template_sc_left = kwant.continuum.discretize(
        template_strings['ham_sc_left'], **kwargs)
    template_sc_right = kwant.continuum.discretize(
        template_strings['ham_sc_right'], **kwargs)

    def union_shape(shapes):
        def _shape(pos):
            res = False
            for shape in shapes:
                res |= shape(pos)
            return res
        return _shape
        
    def intersection_shape(shape_A, shape_B):
        def _shape(pos):
            return shape_A(pos) and not shape_B(pos)
        return _shape
    
    def below_zigzag(z_x, z_y, offset):
        def shape(site):
            x, y = site.pos - offset
            if 0 <= x < z_x:
                return y < x * z_y / z_x
            if z_x <= x < 2*z_x:
                return y < z_y - (x - z_x) * z_y / z_x
            else:
                return False
        return shape
    
    def above_zigzag(z_x, z_y, offset):
        def shape(pos):
            return not below_zigzag(z_x, z_y, offset)(pos)
        return shape
    
    def within_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, offset)(pos) and 0<=x<L_x
        return shape
    
    def edge_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return ((below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1] + L_m-a*edge_thickness))(pos) and 0<=x<L_x)
                    or
                   (below_zigzag(z_x, z_y, (offset[0], offset[1] +edge_thickness*a))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1]))(pos) and 0<=x<L_x))
        return shape
    
    number_of_zigzags = int(L_x // (2*z_x))+1
    within_zigzag_shapes = [within_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_zigzag_shapes = [edge_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_shape =union_shape(edge_zigzag_shapes)
    
    middle_shape_with_edge = union_shape(within_zigzag_shapes)
    middle_shape = intersection_shape(middle_shape_with_edge, edge_shape)
    
    def top_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and 0 <= y < L_m + z_y + W_up
    top_shape_with_some_down = intersection_shape(top_shape_block, middle_shape_with_edge)
    
    def down_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and -W_down <= y < L_m + z_y
    down_shape_with_some_top = intersection_shape(down_shape_block, middle_shape_with_edge)
    
    top_shape = top_shape_with_some_down
#     intersection_shape(top_shape_with_some_down, down_shape_with_some_top)
    down_shape = down_shape_with_some_top
#     intersection_shape(down_shape_with_some_top, top_shape)
    
    # BUILD FINITE SYSTEM
    syst = kwant.Builder()
    syst.fill(template_normal, middle_shape, (0, L_m//2))
    
    if edge_thickness == 1:
        for x in np.arange(0, L_x, a):
            y = (x%z_x)*z_y/z_x
            if (x//z_x)%2 == 1:
                y = z_y - y

            syst.fill(template_barrier, edge_shape, (x, y))
            syst.fill(template_barrier, edge_shape, (x, y + L_m-a))
    
    else:  
        syst.fill(template_barrier, edge_shape, (0, 0))
        syst.fill(template_barrier, edge_shape, (0, L_m-a))
    
    syst.fill(template_sc_left, top_shape, (0, L_m))
    syst.fill(template_sc_right, down_shape, (0, -a))
    
    syst = syst.finalized()

    return syst

In [ ]:

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = .58,
                 phase = 3.14,
                 T = 0.0,
                 V = 13.0)

syst_pars = dict(a=10, L_m=300, L_x=2450, W_up=200, W_down=200, z_x=350, z_y=350,
                 offset_lead=100, k_x_in_sc=True, leaded=False)
syst_pars = dict(a=10, L_m=400, L_x=8000, W_up=200, W_down=200, z_x=350, z_y=0,
                 offset_lead=100, k_x_in_sc=True, leaded=False)

params = dict(**constants,
              **params_raw)

In [ ]:
import plotting_results
syst = l_shaped_system(**syst_pars)
ax1=plotting_results.plot_syst(syst=syst, params=params);
# ax1.delaxes(ax1.axes[1])
# ax1.set_size_inches(20, 10)
# ax1.savefig('L_system')

In [ ]:
import formulas as f
vf = f.fermi_velocity(10, 0.02)
f.thouless_energy(vf, 400)/26/params['mu_B']


In [ ]:
_syst_pars = dict(a=5, L_m=400, L_x=1000, L_up=200, L_down=200,
                  k_x_in_sc=True,)

_=sns_system.make_ns_junction(**_syst_pars)

import scattering
scattering.transparency(_, params)

In [ ]:
ham = syst.hamiltonian_submatrix(params=params, sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.sparse_diag(ham,4,0)
e

In [ ]:
import matplotlib.pyplot as plt
ax=plt.subplot(111)
_ev = np.abs(d(ev[:,0]))
kwant.plotter.map(syst, _ev, ax=ax);
ax1=plotting_results.plot_syst(syst=syst, params=params, ax=ax);
# plt.savefig('test',dpi=800)

In [ ]:
array([ -6.93692313e-05,   6.93692313e-05,  -8.59470886e-02,
         8.59470886e-02])

In [ ]:
supercurrent.fermi_dirac(8.6e-2, {'k':params['k'], 'T':40e-3})

In [ ]:
import matplotlib.pyplot as plt
ax=plt.subplot(111)
_ev = np.abs(d(ev[:,0]))
kwant.plotter.map(syst, _ev, ax=ax);
ax1=plotting_results.plot_syst(syst=syst, params=params, ax=ax);
# plt.savefig('test',dpi=800)

In [ ]:
array([-0.02970037,  0.02970037,  0.09058324, -0.09058324])

In [ ]:
hv.extension('matplotlib')

In [ ]:
%%output filename='./first_and_second_energy_table' fig='png'
distributed_sns.ptable(params_raw) + distributed_sns.ptable(syst_pars)

In [ ]:
def lowest_two(pB):    
    _params = params.copy()
    _params['phase'] = pB[0]
    _params['B'] = pB[1]
    
    ham = syst.hamiltonian_submatrix(params=_params, sparse=True)
    d = kwant.operator.Density(syst)
    e, ev = spectrum.sparse_diag(ham,4,0)
    return np.sort(e)[2:4]


In [ ]:
import adaptive
adaptive.notebook_extension()

In [ ]:
learner = adaptive.Learner2D(lowest_two, [[0,2*np.pi], [0,1.5]])
def lowest_two1d(B):
    return lowest_two((np.pi, B))
learner = adaptive.Learner1D(lowest_two1d, [0,4.5])

In [ ]:
runner = adaptive.Runner(learner, ntasks=30)
runner.live_info()

In [ ]:
runner.task.result()

In [ ]:
hv.extension('bokeh')

In [ ]:
learner.plot()

In [ ]:
_=learner.plot(800)

In [ ]:
a=_.items()[1][1].Image.I

In [ ]:
hv.operation.contours(a, levels=[0.1 ,0.21])

In [ ]:
%%opts Image [colorbar=True]
# %%output filename='./first_and_second_energy_zigzag_400nm' fig='png'
hv.HoloMap(learner.plot(800).redim(x='phase', y='B'), kdims='nth lowest eigenvalue')


In [ ]:
ax.savefig('snake_major')

In [ ]:
from multiprocessing import Pool

In [ ]:
p=Pool(12)

In [ ]:
def G(e):
    smat = kwant.smatrix(syst, energy=e, params=params)
    return smat.conductance_matrix()[0][0]

In [ ]:
G(.11)

In [ ]:
res = p.map(G, np.linspace(-.2, .2, 101))